#**Convexity Trading and Bond Portfolio Risk Management**

##**Function for discount factors from spot rates**

In [ ]:
def disc_fact(spot):
  discount_factors=[]
  for i in range(len(spot)):
    discount_factors.append(1/(1+spot[i])**(i+1))
  return discount_factors

In [ ]:
#Execute the above function with an example
spot_rates = [0.007, 0.015, 0.023, 0.030, 0.037, 0.043, 0.048, 0.052, 0.056, 0.06]
spot_rates

disc_fact(spot_rates)

[0.99304865938431,
 0.9706617486471405,
 0.934056396477656,
 0.8884870479156888,
 0.8338851080567814,
 0.7767730500853498,
 0.7202296919196319,
 0.666613464512311,
 0.612385270513089,
 0.5583947769151179]

##**Function for bond price from spot rates**

In [ ]:
def bond_price_spot(cpn, spot, tenor, principal):
  price = 0
  z = disc_fact(spot)
  for i in range(tenor):
    if(i == tenor-1):
      price = price +  principal*(1+cpn)*z[i]
    else:
      price = price +  principal*cpn*z[i]
  return price

In [ ]:
bond_price_spot(0.1,spot_rates,5,100)

129.58990041049393

##**Function for bond prices from YTM**

In [ ]:
def bond_price_ytm(cpn, ytm, tenor, principal):
  return ((1-(1+ytm)**(-1*tenor))*(cpn*principal/ytm)) + (principal*((1+ytm)**(-1*tenor)))

In [ ]:
bond_price_ytm(0.05, 0.05, 5, 100)

100.0

##**YTM Calculation using Newton Raphson**

### 1. Finite difference for first derivative calculation for Newton Raphson

In [ ]:
def func_val(cpn, ytm, tenor, principal, fair_price):
  return bond_price_ytm(cpn, ytm, tenor, principal) - fair_price

def func_deriv(cpn, ytm, tenor, principal, fair_price, shock_size):
  up_shock_output = func_val(cpn, ytm+shock_size, tenor, principal, fair_price)
  down_shock_output = func_val(cpn, ytm-shock_size, tenor, principal, fair_price)
  return ((up_shock_output - down_shock_output)/(2*shock_size))

###Python function for Newton Raphson for YTM calculation

In [ ]:
def newtonRaphson_ytm(guess_ytm, max_iter, tol_val, cpn, tenor, principal, shock_size, fair_price):
  nb_iter = 0
  diff = func_val(cpn, guess_ytm, tenor, principal, fair_price)/func_deriv(cpn, guess_ytm, tenor, principal, fair_price, shock_size)
  if(abs(diff)>tol_val):
    for i in range(max_iter):
      if(abs(diff)>tol_val):
        guess_ytm = guess_ytm - diff
        diff = func_val(cpn, guess_ytm, tenor, principal, fair_price)/func_deriv(cpn, guess_ytm, tenor, principal, fair_price, shock_size)
        nb_iter+=1 #nb_iter = nb_iter +1
      else:
        exit
    #else:
      #print("The value of the root is : ", "%.5f"% guess_ytm)
      #return 
    
    #print("The value of the root is : ", "%.5f"% guess_ytm)
  return guess_ytm 

In [ ]:
newtonRaphson_ytm(0.1165, 1000, 0.000001, 0.09, 8, 100, 0.001, 111.94259701242748)

0.06999965735529554

##**Functions for modified duration and modified convexity**

### 1. Modified Duration

In [ ]:
def bond_modified_duration(cpn, ytm, tenor, principal, shock_size):
  base_price = bond_price_ytm(cpn, ytm, tenor, principal)
  up_shock_output = bond_price_ytm(cpn, ytm+shock_size, tenor, principal)
  down_shock_output = bond_price_ytm(cpn, ytm-shock_size, tenor, principal)
  return ((down_shock_output-up_shock_output)/(2*base_price*shock_size))

In [ ]:
bond_modified_duration(0.09, 0.15, 5, 100, 0.001)

3.600034447830464

### 2. Modified Convexity 

In [ ]:
def bond_modified_convexity(cpn, ytm, tenor, principal, shock_size):
  up_shock_output = bond_price_ytm(cpn, ytm+shock_size, tenor, principal)
  down_shock_output = bond_price_ytm(cpn, ytm-shock_size, tenor, principal)
  base_price = bond_price_ytm(cpn, ytm, tenor, principal)
  return ((down_shock_output+up_shock_output-2*base_price)/(base_price*shock_size*shock_size))

In [ ]:
bond_modified_convexity(0.09, 0.07, 5, 100, 0.001)

21.15735693606324

##**Portfolio Construction for convexity trade**

### 1. Calculation of bond price and bond YTM for all the bonds in the portfolio

In [ ]:
cpn_5Y = float(input("Enter Coupon for 5Y bond: "))
cpn_2Y = float(input("Enter Coupon for 2Y bond: "))
cpn_8Y = float(input("Enter Coupon for 8Y bond: "))

#Note, the total principal for the bond = number of bonds * principal of 1 bond
prin_5Y = float(input("Enter Total Princpal for 5Y bond: "))
prin_2Y = float(input("Enter Total Princpal for 2Y bond: "))
prin_8Y = float(input("Enter Total Princpal for 8Y bond: "))

#cpn_5Y = 0.1
#cpn_2Y = 0.07
#cpn_8Y = 0.14

#prin_5Y = 100
#prin_2Y = 100
#prin_8Y = 100

bond_price_5Y = bond_price_spot(cpn_5Y,spot_rates,5,prin_5Y)
bond_price_2Y = bond_price_spot(cpn_2Y,spot_rates,2,prin_2Y)
bond_price_8Y = bond_price_spot(cpn_8Y,spot_rates,8,prin_8Y)

Enter Coupon for 5Y bond: 0.10
Enter Coupon for 2Y bond: 0.07
Enter Coupon for 8Y bond: 0.14
Enter Total Princpal for 5Y bond: 100
Enter Total Princpal for 2Y bond: 100
Enter Total Princpal for 8Y bond: 100


In [ ]:
print(bond_price_5Y)
print(bond_price_2Y)
print(bond_price_8Y)

129.58990041049393
110.8121477209342
161.63391878921527


In [ ]:
bond_ytm_5Y = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_5Y, 5, prin_5Y, 0.001, 129.58990041049393)
bond_ytm_2Y = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_2Y, 2, prin_2Y, 0.001, 110.8121477209342)
bond_ytm_8Y = newtonRaphson_ytm(0.2, 1000, 0.000001, cpn_8Y, 8, prin_8Y, 0.001, 161.63391878921527)

In [ ]:
print(bond_ytm_5Y)
print(bond_ytm_2Y)
print(bond_ytm_8Y)

0.034547791599101124
0.014740982517686735
0.04612642118374499


### 2.Calculation of dollar duration and dollar convexity of all the bonds in the portfolio

Modified Duration = %change in portfolio value due 1% change in yield corresponding to first order sensitivity
Duration = -(1/P)*(dP/dY)

Dollar Duration = Dollar change in the value of portfolio for 1% change in yield based on first order sensitivity
Dollar Duration = Duration * Portfolio Value

Modified Convexity = %change in portfolio value due 1% change in yield corresponding to second order sensitivity
Duration = (1/P)*(d2P/dY2)

Dollar Convexity = Dollar change in the value of portfolio for 1% change in yield  based on second order sensitivity
Dollar Duration = Convexity * Portfolio Value

In [ ]:
dollar_duration_5Y = bond_modified_duration(cpn_5Y, bond_ytm_5Y, 5, prin_5Y, 0.001)*bond_price_5Y
dollar_duration_2Y = bond_modified_duration(cpn_2Y, bond_ytm_2Y, 2, prin_2Y, 0.001)*bond_price_2Y
dollar_duration_8Y = bond_modified_duration(cpn_8Y, bond_ytm_8Y, 8, prin_8Y, 0.001)*bond_price_8Y

In [ ]:
print(dollar_duration_5Y)
print(dollar_duration_2Y)
print(dollar_duration_8Y)

535.9506832881415
211.60709719788346
907.7798910829376


In [ ]:
dollar_convexity_5Y = bond_modified_convexity(cpn_5Y, bond_ytm_5Y, 5, prin_5Y, 0.001)*bond_price_5Y
dollar_convexity_2Y = bond_modified_convexity(cpn_2Y, bond_ytm_2Y, 2, prin_2Y, 0.001)*bond_price_2Y
dollar_convexity_8Y = bond_modified_convexity(cpn_8Y, bond_ytm_8Y, 8, prin_8Y, 0.001)*bond_price_8Y

In [ ]:
print(dollar_convexity_5Y)
print(dollar_convexity_2Y)
print(dollar_convexity_8Y)

2936.5547129816155
618.8997934422958
6909.778941967457
